# TFRecord 数据格式详解

TFRecord 是 TensorFlow 生态系统中用于高效存储和读取大规模数据集的二进制文件格式。本教程从原理到实践系统讲解 TFRecord 的核心概念。

## 核心知识点

1. TFRecord 格式的设计原理与性能优势
2. 基于 Protocol Buffers 的数据序列化机制
3. 压缩策略与 I/O 性能权衡
4. 生产环境中的分片与并行读取策略

## 1. TFRecord 设计原理

### 1.1 解决的核心问题

在大规模机器学习场景中，数据 I/O 常成为训练瓶颈。考虑 ImageNet 数据集：

- **120 万张图像**意味着 120 万次文件系统调用
- 每次 `open() -> read() -> close()` 都涉及磁盘寻道
- 机械硬盘寻道时间约 10ms，SSD 约 0.1ms，但频繁小文件访问仍是瓶颈

TFRecord 的解决方案：**将大量小文件合并为少量大文件**。

### 1.2 文件结构

TFRecord 文件由连续的记录（Record）组成，每条记录的二进制结构：

```
[length: uint64][length_crc: uint32][data: byte[length]][data_crc: uint32]
```

- `length`: 数据长度（8 字节）
- `length_crc`: 长度字段的 CRC32 校验（4 字节）
- `data`: 实际数据内容
- `data_crc`: 数据的 CRC32 校验（4 字节）

CRC 校验确保数据在存储和传输过程中的完整性。

In [ ]:
import tensorflow as tf
import numpy as np
import os
import tempfile
from pathlib import Path

# 环境信息
print(f"TensorFlow 版本: {tf.__version__}")
print(f"Eager 模式: {tf.executing_eagerly()}")

# 创建临时工作目录
WORK_DIR = Path(tempfile.mkdtemp(prefix="tfrecord_demo_"))
print(f"工作目录: {WORK_DIR}")

## 2. TFRecord 基础读写

### 2.1 写入原始字节数据

`TFRecordWriter` 接受任意字节序列。在底层，它会自动添加长度前缀和 CRC 校验。

In [ ]:
# 基础写入示例
basic_tfrecord = WORK_DIR / "basic_example.tfrecord"

# 准备测试数据
records = [
    b"TFRecord stores arbitrary binary data",
    b"Each record is length-prefixed with CRC32 checksum",
    b"Sequential access pattern optimizes disk I/O",
]

# 使用上下文管理器确保资源正确释放
with tf.io.TFRecordWriter(str(basic_tfrecord)) as writer:
    for record in records:
        writer.write(record)

# 验证写入结果
file_size = basic_tfrecord.stat().st_size
raw_size = sum(len(r) for r in records)
overhead = file_size - raw_size

print(f"原始数据: {raw_size} bytes")
print(f"文件大小: {file_size} bytes")
print(f"格式开销: {overhead} bytes ({overhead / len(records):.1f} bytes/record)")

### 2.2 读取 TFRecord

`TFRecordDataset` 返回 `tf.data.Dataset` 对象，可无缝集成到训练流水线中。

In [ ]:
# 创建数据集对象
dataset = tf.data.TFRecordDataset(str(basic_tfrecord))

# 检查数据集规格
print(f"元素类型: {dataset.element_spec.dtype}")
print(f"元素形状: {dataset.element_spec.shape}")
print()

# 遍历读取
for idx, raw_record in enumerate(dataset):
    # 返回的是 tf.Tensor，dtype=tf.string（表示字节序列）
    content = raw_record.numpy().decode("utf-8")
    print(f"[{idx}] {content}")

## 3. 压缩 TFRecord

### 3.1 压缩的适用场景

| 场景 | 推荐策略 |
|------|----------|
| 网络传输（云存储） | GZIP，最大化压缩比 |
| 本地 SSD | 不压缩或 ZLIB，避免 CPU 成为瓶颈 |
| 高重复数据（如文本） | GZIP，压缩收益大 |
| 已压缩数据（如 JPEG） | 不压缩，二次压缩收益小 |

### 3.2 压缩格式对比

- **GZIP**: 压缩比高（通常 60-80%），解压较慢
- **ZLIB**: 压缩比与 GZIP 接近，解压稍快
- **不压缩**: 零 CPU 开销，适合 SSD + 低重复数据

In [ ]:
# 生成具有重复模式的测试数据（模拟文本或结构化数据）
test_records = [
    b"feature_vector:" + np.random.bytes(100) + b"label:0" * 50,
    b"feature_vector:" + np.random.bytes(100) + b"label:1" * 50,
    b"feature_vector:" + np.random.bytes(100) + b"label:2" * 50,
]

# 定义三种写入配置
configs = {
    "uncompressed": None,
    "gzip": tf.io.TFRecordOptions(compression_type="GZIP"),
    "zlib": tf.io.TFRecordOptions(compression_type="ZLIB"),
}

results = {}
for name, options in configs.items():
    filepath = WORK_DIR / f"compression_{name}.tfrecord"
    with tf.io.TFRecordWriter(str(filepath), options=options) as writer:
        for record in test_records:
            writer.write(record)
    results[name] = filepath.stat().st_size

# 输出对比结果
raw_size = sum(len(r) for r in test_records)
print(f"原始数据大小: {raw_size} bytes\n")
print("压缩效果对比:")
print("-" * 45)
for name, size in results.items():
    ratio = (1 - size / raw_size) * 100
    print(f"{name:15} | {size:6} bytes | 压缩率 {ratio:5.1f}%")

In [ ]:
# 读取压缩文件时必须指定相同的压缩类型
gzip_path = WORK_DIR / "compression_gzip.tfrecord"

# 正确方式：指定压缩类型
compressed_ds = tf.data.TFRecordDataset(
    str(gzip_path),
    compression_type="GZIP"
)

print("读取 GZIP 压缩文件:")
for idx, record in enumerate(compressed_ds):
    print(f"[{idx}] 长度: {len(record.numpy())} bytes")

## 4. Protocol Buffers 与 tf.train.Example

### 4.1 结构化数据的挑战

原始字节流无法表达复杂的数据结构。例如，一个图像分类样本需要存储：

- 图像原始像素（或编码后的 JPEG/PNG）
- 图像尺寸（高度、宽度、通道数）
- 分类标签
- 元数据（文件名、采集时间等）

**Protocol Buffers** (Protobuf) 是 Google 开发的高效序列化框架，TensorFlow 基于它定义了 `tf.train.Example` 消息类型。

### 4.2 tf.train.Feature 数据类型

| 类型 | Python 对应 | 典型用途 |
|------|-------------|----------|
| `BytesList` | `bytes`, `str` | 图像、音频、文本 |
| `FloatList` | `float`, `np.float32` | 特征向量、权重 |
| `Int64List` | `int`, `np.int64` | 标签、索引、计数 |

In [ ]:
def bytes_feature(value):
    """将字节或字符串转换为 BytesList Feature"""
    if isinstance(value, str):
        value = value.encode("utf-8")
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def float_feature(value):
    """将浮点数或数组转换为 FloatList Feature"""
    if isinstance(value, np.ndarray):
        value = value.flatten().tolist()
    elif not isinstance(value, (list, tuple)):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def int64_feature(value):
    """将整数或数组转换为 Int64List Feature"""
    if isinstance(value, np.ndarray):
        value = value.flatten().tolist()
    elif not isinstance(value, (list, tuple)):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


# 验证辅助函数
print("BytesList:", bytes_feature("hello"))
print("FloatList:", float_feature([1.0, 2.0, 3.0]))
print("Int64List:", int64_feature(42))

### 4.3 构建 tf.train.Example

`tf.train.Example` 本质上是一个特征字典，键为字符串，值为 `tf.train.Feature`。

In [ ]:
# 模拟图像分类样本
np.random.seed(42)

sample_image = np.random.randint(0, 256, size=(32, 32, 3), dtype=np.uint8)
sample_label = 5
sample_id = "img_00001"
sample_embedding = np.random.randn(128).astype(np.float32)

# 构建 Example
example = tf.train.Example(
    features=tf.train.Features(
        feature={
            # 图像数据：序列化为字节存储
            "image/encoded": bytes_feature(sample_image.tobytes()),
            "image/height": int64_feature(sample_image.shape[0]),
            "image/width": int64_feature(sample_image.shape[1]),
            "image/channels": int64_feature(sample_image.shape[2]),
            # 标签
            "label": int64_feature(sample_label),
            # 元数据
            "sample_id": bytes_feature(sample_id),
            # 预计算的特征向量
            "embedding": float_feature(sample_embedding),
        }
    )
)

# 序列化为字节
serialized = example.SerializeToString()
print(f"序列化后大小: {len(serialized)} bytes")
print(f"原始数据大小: {sample_image.nbytes + sample_embedding.nbytes} bytes")

In [ ]:
# 批量写入多个样本
structured_tfrecord = WORK_DIR / "structured_samples.tfrecord"
NUM_SAMPLES = 100

with tf.io.TFRecordWriter(str(structured_tfrecord)) as writer:
    for i in range(NUM_SAMPLES):
        # 生成随机样本
        img = np.random.randint(0, 256, (32, 32, 3), dtype=np.uint8)
        label = np.random.randint(0, 10)
        emb = np.random.randn(128).astype(np.float32)
        
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "image/encoded": bytes_feature(img.tobytes()),
                    "image/height": int64_feature(32),
                    "image/width": int64_feature(32),
                    "image/channels": int64_feature(3),
                    "label": int64_feature(label),
                    "sample_id": bytes_feature(f"sample_{i:05d}"),
                    "embedding": float_feature(emb),
                }
            )
        )
        writer.write(example.SerializeToString())

file_size_kb = structured_tfrecord.stat().st_size / 1024
print(f"写入 {NUM_SAMPLES} 个样本")
print(f"文件大小: {file_size_kb:.1f} KB")
print(f"平均每样本: {file_size_kb * 1024 / NUM_SAMPLES:.0f} bytes")

### 4.4 解析 tf.train.Example

解析时需要提供与写入一致的特征描述（Feature Description），描述每个特征的类型和形状。

**关键概念**：
- `FixedLenFeature`: 固定长度特征，缺失时报错
- `VarLenFeature`: 变长特征，返回 SparseTensor
- `FixedLenSequenceFeature`: 固定长度的序列特征

In [ ]:
# 定义特征描述
feature_description = {
    "image/encoded": tf.io.FixedLenFeature([], tf.string),
    "image/height": tf.io.FixedLenFeature([], tf.int64),
    "image/width": tf.io.FixedLenFeature([], tf.int64),
    "image/channels": tf.io.FixedLenFeature([], tf.int64),
    "label": tf.io.FixedLenFeature([], tf.int64),
    "sample_id": tf.io.FixedLenFeature([], tf.string),
    "embedding": tf.io.FixedLenFeature([128], tf.float32),
}


def parse_example(serialized_example):
    """解析单个序列化的 Example"""
    parsed = tf.io.parse_single_example(serialized_example, feature_description)
    
    # 解码图像数据
    height = tf.cast(parsed["image/height"], tf.int32)
    width = tf.cast(parsed["image/width"], tf.int32)
    channels = tf.cast(parsed["image/channels"], tf.int32)
    
    image = tf.io.decode_raw(parsed["image/encoded"], tf.uint8)
    image = tf.reshape(image, [height, width, channels])
    
    # 归一化到 [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    return {
        "image": image,
        "embedding": parsed["embedding"],
        "sample_id": parsed["sample_id"],
    }, parsed["label"]

In [ ]:
# 构建高效的数据流水线
BATCH_SIZE = 16

dataset = (
    tf.data.TFRecordDataset(str(structured_tfrecord))
    .map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=100)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

# 验证流水线输出
print("数据集元素规格:")
print(dataset.element_spec)
print()

for features, labels in dataset.take(1):
    print(f"图像批次: {features['image'].shape}")
    print(f"嵌入批次: {features['embedding'].shape}")
    print(f"标签批次: {labels.shape}")
    print(f"图像值域: [{features['image'].numpy().min():.3f}, {features['image'].numpy().max():.3f}]")

## 5. 生产环境最佳实践：分片存储

### 5.1 为什么需要分片

单个超大 TFRecord 文件存在以下问题：

1. **并行读取受限**: 单文件只能顺序读取
2. **故障恢复困难**: 文件损坏可能丢失全部数据
3. **分布式训练**: 多 Worker 无法高效分配数据

### 5.2 分片策略

**推荐配置**：
- 每个分片 100-200 MB
- 分片数量 >= Worker 数量 × 10（确保负载均衡）
- 文件命名: `data-00000-of-00100.tfrecord`

In [ ]:
# 分片写入示例
NUM_SHARDS = 4
SAMPLES_PER_SHARD = 25

shard_paths = []
for shard_id in range(NUM_SHARDS):
    shard_name = f"data-{shard_id:05d}-of-{NUM_SHARDS:05d}.tfrecord"
    shard_path = WORK_DIR / shard_name
    shard_paths.append(str(shard_path))
    
    with tf.io.TFRecordWriter(str(shard_path)) as writer:
        for i in range(SAMPLES_PER_SHARD):
            global_idx = shard_id * SAMPLES_PER_SHARD + i
            img = np.random.randint(0, 256, (32, 32, 3), dtype=np.uint8)
            label = np.random.randint(0, 10)
            
            example = tf.train.Example(
                features=tf.train.Features(
                    feature={
                        "image/encoded": bytes_feature(img.tobytes()),
                        "image/height": int64_feature(32),
                        "image/width": int64_feature(32),
                        "image/channels": int64_feature(3),
                        "label": int64_feature(label),
                        "sample_id": bytes_feature(f"sample_{global_idx:05d}"),
                    }
                )
            )
            writer.write(example.SerializeToString())
    
    print(f"分片 {shard_id}: {shard_path.stat().st_size / 1024:.1f} KB")

print(f"\n共创建 {NUM_SHARDS} 个分片，{NUM_SHARDS * SAMPLES_PER_SHARD} 个样本")

### 5.3 使用 interleave 并行读取

`interleave` 操作从多个数据源交错读取，最大化 I/O 吞吐量。

**关键参数**：
- `cycle_length`: 同时读取的文件数
- `num_parallel_calls`: 并行处理的线程数
- `deterministic`: 设为 False 可提升性能（但结果不可复现）

In [ ]:
# 简化的解析函数（仅用于分片数据）
shard_feature_desc = {
    "image/encoded": tf.io.FixedLenFeature([], tf.string),
    "image/height": tf.io.FixedLenFeature([], tf.int64),
    "image/width": tf.io.FixedLenFeature([], tf.int64),
    "image/channels": tf.io.FixedLenFeature([], tf.int64),
    "label": tf.io.FixedLenFeature([], tf.int64),
    "sample_id": tf.io.FixedLenFeature([], tf.string),
}


def parse_shard_example(serialized):
    """解析分片数据"""
    parsed = tf.io.parse_single_example(serialized, shard_feature_desc)
    
    image = tf.io.decode_raw(parsed["image/encoded"], tf.uint8)
    image = tf.reshape(image, [32, 32, 3])
    image = tf.cast(image, tf.float32) / 255.0
    
    return image, parsed["label"]


# 构建分片读取流水线
files_dataset = tf.data.Dataset.from_tensor_slices(shard_paths)

# 文件级打乱
files_dataset = files_dataset.shuffle(buffer_size=NUM_SHARDS)

# 并行交错读取
sharded_dataset = files_dataset.interleave(
    lambda filepath: tf.data.TFRecordDataset(filepath),
    cycle_length=NUM_SHARDS,
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False
)

# 完整流水线
sharded_dataset = (
    sharded_dataset
    .map(parse_shard_example, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=100)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)

# 验证
total_samples = 0
for images, labels in sharded_dataset:
    total_samples += images.shape[0]

print(f"从 {NUM_SHARDS} 个分片读取 {total_samples} 个样本")

## 6. 实战：MNIST 数据集转换

演示完整的数据集转换流程。

In [ ]:
# 加载 MNIST（使用子集进行演示）
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 取前 200 个样本用于快速演示
x_demo, y_demo = x_train[:200], y_train[:200]

print(f"演示数据: {x_demo.shape}, 标签分布: {np.bincount(y_demo)}")

In [ ]:
def create_mnist_example(image, label):
    """创建 MNIST 样本的 tf.train.Example"""
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "image": bytes_feature(image.tobytes()),
                "label": int64_feature(int(label)),
                "height": int64_feature(image.shape[0]),
                "width": int64_feature(image.shape[1]),
            }
        )
    )


# 写入 TFRecord
mnist_tfrecord = WORK_DIR / "mnist_demo.tfrecord"

with tf.io.TFRecordWriter(str(mnist_tfrecord)) as writer:
    for img, lbl in zip(x_demo, y_demo):
        example = create_mnist_example(img, lbl)
        writer.write(example.SerializeToString())

print(f"MNIST TFRecord: {mnist_tfrecord.stat().st_size / 1024:.1f} KB")

In [ ]:
# MNIST 解析函数
mnist_feature_desc = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "label": tf.io.FixedLenFeature([], tf.int64),
    "height": tf.io.FixedLenFeature([], tf.int64),
    "width": tf.io.FixedLenFeature([], tf.int64),
}


def parse_mnist(serialized):
    """解析 MNIST TFRecord"""
    parsed = tf.io.parse_single_example(serialized, mnist_feature_desc)
    
    image = tf.io.decode_raw(parsed["image"], tf.uint8)
    image = tf.reshape(image, [28, 28, 1])
    image = tf.cast(image, tf.float32) / 255.0
    
    return image, parsed["label"]


# 构建训练流水线
mnist_dataset = (
    tf.data.TFRecordDataset(str(mnist_tfrecord))
    .map(parse_mnist, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(200)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

# 验证
for images, labels in mnist_dataset.take(1):
    print(f"图像: {images.shape}, 标签: {labels.shape}")
    print(f"值域: [{images.numpy().min():.3f}, {images.numpy().max():.3f}]")

## 7. 清理资源

In [ ]:
import shutil

shutil.rmtree(WORK_DIR)
print(f"已清理: {WORK_DIR}")

## 总结

### 核心要点

1. **TFRecord 本质**: 长度前缀 + CRC 校验的二进制记录序列
2. **Protocol Buffers**: `tf.train.Example` 提供结构化数据的标准表示
3. **压缩策略**: 根据数据类型和部署环境选择 GZIP/ZLIB/不压缩
4. **分片存储**: 生产环境必备，支持并行读取和故障隔离

### 性能优化清单

| 优化点 | 方法 |
|--------|------|
| I/O 瓶颈 | 分片 + `interleave()` 并行读取 |
| CPU 瓶颈 | `map(num_parallel_calls=AUTOTUNE)` |
| GPU 空闲 | `prefetch(AUTOTUNE)` |
| 内存不足 | 流式处理，避免一次性加载 |

### 参考文档

- [TFRecord 官方指南](https://www.tensorflow.org/tutorials/load_data/tfrecord)
- [tf.data 性能优化](https://www.tensorflow.org/guide/data_performance)